# Tutorial of the mitigation
Before reading this tutorial, we suggest you read the sampling tutorial.

In [1]:
import sys

sys.path.append("../")
from utils.challenge_2024 import ChallengeSampling
from quri_parts.core.measurement import bitwise_commuting_pauli_measurement
from quri_parts.core.sampling.shots_allocator import create_equipartition_shots_allocator
from quri_parts.core.state import GeneralCircuitQuantumState

# define challenge_sampling
challenge_sampling = ChallengeSampling()

   Resolving package versions...
  No Changes to `~/.julia/environments/pyjuliapkg/Project.toml`
  No Changes to `~/.julia/environments/pyjuliapkg/Manifest.toml`


test mps sampling took: (24.094444036483765, Counter({2: 5, 0: 5}))


# Prepare a circuit

In [2]:
from math import pi
from quri_parts.circuit import QuantumCircuit
# circuit with 4 qubits
circuit = QuantumCircuit(4)
circuit.add_X_gate(0)
circuit.add_H_gate(1)
circuit.add_Y_gate(2)
circuit.add_CNOT_gate(1, 2)
circuit.add_RX_gate(3, pi/4)

# circuit state
circuit_state = GeneralCircuitQuantumState(4, circuit)

# Define sampler and sampling_estilator

In [3]:
# create concurrent sampler
concurrent_sampler = challenge_sampling.create_concurrent_sampler()
# define some inputs for the sampling_estimator
shots_allocator = create_equipartition_shots_allocator()
measurement_factory = bitwise_commuting_pauli_measurement
n_shots = 10000

# create concurrent sampling estimator
concurrent_estimator = challenge_sampling.create_sampling_concurrent_estimator(
    n_shots, measurement_factory, shots_allocator
)

   Resolving package versions...
  No Changes to `~/.julia/environments/pyjuliapkg/Project.toml`
  No Changes to `~/.julia/environments/pyjuliapkg/Manifest.toml`


# Prepare an operator

In [4]:
# define an operator to be estimated

from quri_parts.core.operator import Operator, pauli_label, PAULI_IDENTITY
op = Operator({
    pauli_label("Z0"): 0.25,
    pauli_label("Z1 Z2"): 2.0,
    pauli_label("X1 X2"): 0.5,
    pauli_label("Z1 Y3"): 1.0,
    pauli_label("Z2 Y3"): 1.5,
    pauli_label("X1 Y3"): 2.0,
    PAULI_IDENTITY: 3.0,
})
print(op)

0.25*Z0 + 2.0*Z1 Z2 + 0.5*X1 X2 + 1.0*Z1 Y3 + 1.5*Z2 Y3 + 2.0*X1 Y3 + 3.0*I


# Examples of mitigation
In this tutorial, we will see how to use three mitigation methods: Zero Noise Extrapolation (ZNE) and Readout mitigation. These mitigations are implemented in QURI Parts, and although we will not described here, the Clifford Data Regression (CDR) is also implemented. So you can use without any implementation by yourself.

## ZNE

In [5]:
from quri_parts.algo.mitigation.zne import (
    create_zne_estimator,
    create_folding_left,
    create_polynomial_extrapolate,
)

# choose an extrapolation method
extrapolate_method = create_polynomial_extrapolate(order=2)
# choose how folding your circuit
folding_method = create_folding_left()
# define scale factors
scale_factors = [1, 3, 5]

# construct estimator by using zne (only concurrent estimator can be used)
zne_estimator = create_zne_estimator(
    concurrent_estimator, scale_factors, extrapolate_method, folding_method
)
# by using this estimator, you can obtain an estimated value with ZNE
zne_estimated_value = zne_estimator(op, circuit_state)

print(f"zne_estimated_value :{zne_estimated_value.value} ")

zne_estimated_value :1.1077500000000016 


## Readout 

In [6]:
from quri_parts.algo.mitigation.readout_mitigation import (
    create_readout_mitigation_concurrent_sampler,
    create_readout_mitigation_sampler,
)

# sampler by using readout mitigation (only concurrent sampler can be used)
readout_sampler = create_readout_mitigation_sampler(
    qubit_count=4, sampler=concurrent_sampler, shots=n_shots
)
readout_counts = readout_sampler(circuit, n_shots)

print(f"readout_counts :{readout_counts} ")

readout_counts :{3: 4237.0, 5: 4314.0, 11: 728.0, 13: 721.0} 


Or you can directly get the expectaion value through the readout mitigation by using `sampling_estimate`.

In [7]:
from quri_parts.core.estimator.sampling import sampling_estimate

readout_concurrent_sampler = create_readout_mitigation_concurrent_sampler(
    qubit_count=4, sampler=concurrent_sampler, shots=n_shots
)

readout_eval = sampling_estimate(
    op, circuit_state, n_shots, readout_concurrent_sampler, measurement_factory, shots_allocator
)

print(f"readout_eval :{readout_eval.value} ")

readout_eval :1.2748 
